# Validation - Coleson Cove Case

Shin et al (2020)

#### Packages

In [1]:
import json
from math import pi, ceil, sin, floor
from iapws.iapws97 import _Region4, _Region2, _Region1
from iapws._iapws import _ThCond, _Viscosity
from constructors import *
from scipy.optimize import root
import numpy as np
import os.path
import sys
sys.path.append("../")
import simulate

from water_properties import density, viscosity, conductivity, heat_capacity
from water_at_saturation_properties import saturation_temperature, vapour_density, vapour_total_compressibility,vaporization_enthalpy


## Preliminar Analysis

#### Constants

In [44]:
g = 9.80665 # m/s2 - gravitational acceleration
pi = pi # pi

#### Heat Exchanger Information

In [45]:
Do = 0.0254 # m external diameter
Di = Do - 2*0.00125 # m external diameter
n_tubes = 6720
arrange = "triangle"
#arrange = "square"

L = 17 # m tube length
W = 1.8 * 2 # m bundle width
ep = 45e-6 # m - roughness
kwall = 16 # W / ( m * K )
pitch = 0.0333 # m
Rows = 31
row_pitch = pitch
tubes_per_width = n_tubes/Rows
ResF= 6.03e-5 # K/W
Vext = L * W ** 2 # shell volume in m3


print("Number of tubes per row are ", tubes_per_width)
print("Number of rows is {number:.{digits}f}".format(number=Rows,digits=0))
print("Number of tubes is {number:.{digits}f}".format(number=Rows*tubes_per_width,digits=0))

Number of tubes per row are  216.7741935483871
Number of rows is 31
Number of tubes is 6720


In [46]:
N_wetters = []
row_index = np.linspace(1,Rows,num=Rows)
for i in row_index:
    if arrange == "triangle":
        N_wetters.append(ceil(i/2)-1)
    else:
        N_wetters = row_index-1
N_wetters = np.array(N_wetters)
fNtub = ((N_wetters+1) ** (5/6) - N_wetters ** (5/6))

In [47]:
# Fouling
kf = 2.941 # W / ( m * K ) (Calcium carbonate)
rhof = 2.71e3 # kg/m3
Df = Di * np.exp(- (ResF* (pi * Di * L)) * 2 * kf / Di)
lf = 1e6*(Di-Df)/2  # um fouling thickness

mf = (Di ** 2 - Df ** 2) * rhof / (4 * Di)

print("Wall internal diameter is {number:.{digits}f}".format(number=Di,digits=4))
print("Fouling internal diameter is {number:.{digits}f}".format(number=Df,digits=6))
print("Fouling thickness is {number:.{digits}f} um".format(number=1e6*(Di-Df)/2,digits=2))
print("Fouling deposit mass is {number:.{digits}f} kg/m2".format(number=mf,digits=6))

Wall internal diameter is 0.0229
Fouling internal diameter is 0.022470
Fouling thickness is 214.85 um
Fouling deposit mass is 0.576787 kg/m2


#### Operational Conditions

In [48]:
m = 3859 # kg/s cooling Water
# Expected
Tout_mean_expected = 26.73 + 273.15
Pvap_expected = 5414# Pa
Tvap_expected = 34.3 + 273.15
Pout_expected = 177000
Tin = 11.58 + 273.15 # K
Pin = 200000 # Pa
Pout = 177000 # Pa
Tout = Tout_mean_expected * np.ones(Rows) # K

In [49]:
Tm_expected = 0.5 * (Tin + Tout_mean_expected) # K - mean cooling water temperature
Pm_expected = 0.5 * (Pin + Pout) # Pa - mean cooling water pressure
water_heat_capacity = heat_capacity(Tm_expected, Pm_expected, simplified = True)

exhaust_steam_density = vapour_density(Pvap_expected, simplified = True)
exhaust_vaporization_heat = vaporization_enthalpy(Pvap_expected, simplified = True)

Q_expected = water_heat_capacity * m * (Tout_mean_expected - Tin)
kvap_expected = Q_expected/exhaust_vaporization_heat


print("Expected heat duty to {number:.{digits}f} MW.".format(number=Q_expected*1e-6, digits=2))
print("Expected steam mass flowrate {number:.{digits}f} kg/s.".format(number=kvap_expected, digits=2))
print("The exhaust steam density is equal to {number:.{digits}f} kg/m3."\
      .format(number=exhaust_steam_density, digits=5))
print("The exhaust steam vaporization heat is equal to {number:.{digits}f} J/kg."\
      .format(number=exhaust_vaporization_heat, digits=0))
#vaporization_enthalpy(Pvap, simplified=True)
#print(Pvap, exhaust_vaporization_heat, vaporization_enthalpy(Pvap, simplified=True))

Expected heat duty to 244.78 MW.
Expected steam mass flowrate 101.17 kg/s.
The exhaust steam density is equal to 0.03802 kg/m3.
The exhaust steam vaporization heat is equal to 2419426 J/kg.


#### Simulation input

In [50]:
# Set
Pvap = 5130 # Pa
Tvap = saturation_temperature(Pvap, simplified = True)
#print(Pvap, Tvap, saturation_temperature(Pvap, simplified=True))

#### Hydraulic Calculations

In [51]:
def calculate_darcy(fD0, ep, D, Re):
    """Calculate the darcy friction factor"""
    def idarcy(ifD, ep, D, Re):
        return ifD + 2. * np.log10(ep / 3.72 / D + (2.51 / Re) * ifD )
    sol = root(idarcy, fD0 ** -0.5, args=(ep, D, Re),)
    return  sol.x[0] ** -2

def calculate_fanning(ep, D, Re):
    """Calculate the fanning friction factor"""
    A = (2.457*np.log(((7/Re)**0.9+0.27*ep/D)**-1))**16
    B = (37530/Re)**16
    ff = 2 *((8/Re)**12 + (A+B)**-1.5) ** (1/12)
    return  ff

#### LMTD

In [52]:
def calculate_int(Tin, Tout, Pin, Pout, m_tube, ep, Di, L):
    
    T = 0.5 * (Tin + Tout)
    P = 0.5 * (Pin + Pout)
    
    # Calculating properties
    int_film_density=density(T, P, simplified = True)
    int_film_heat_capacity = heat_capacity(T, P, simplified = True)
    int_film_conductivity = conductivity(T, P, simplified = True)
    int_film_viscosity = viscosity(T, P, simplified = True)
    int_film_prandtl = int_film_heat_capacity * int_film_viscosity / int_film_conductivity

    v = (m_tube/int_film_density)/(0.25 * pi * Di**2) # m/s
    v_abs = abs(v)

    Re_tube = Di * int_film_density * v_abs / int_film_viscosity

    ff = calculate_fanning(ep, Di, Re_tube)
    fD_tube = 4 * ff
    
    # Calculating Darcy
    int_film_Re = Di * int_film_density * v_abs / int_film_viscosity
    int_film_fD = calculate_darcy(fD_tube, ep, Di, int_film_Re)

    # Calculates the Nussel dimensionless number using Petukhov correlation modified by Gnielinski. See Incropera 4th Edition [8.63]
    nusselt = (int_film_fD / 8.) * (int_film_Re - 1000.) * int_film_prandtl / ( 1. + 12.7 * (int_film_fD / 8.) ** 0.5 * (int_film_prandtl ** (2 / 3) - 1.))

    hint = nusselt * int_film_conductivity / Di # W/m2K
    Resint = 1 / (pi * Di * hint * L) # W/K

    tau = (1/8)*fD_tube*int_film_density*v**2
    hL = 0.5 * fD_tube * v ** 2 /  (Di * g )
    DeltaP = g * int_film_density * hL
    pressure_loss_in_tube = DeltaP * L
    Pout_ = Pin - pressure_loss_in_tube
    
    return hint, Resint, int_film_heat_capacity, Pout_, fD_tube, pressure_loss_in_tube, v, Re_tube


def calculate_ext(Tvap, Pvap, To, fNtub_tube, L):
    
    # Calculating properties
    T = 0.5*(Tvap + To)
    
    g = 9.80665
    
    film_density=density(T, Pvap, simplified = True)
    film_heat_capacity = heat_capacity(T, Pvap, simplified = True)
    film_conductivity = conductivity(T, Pvap, simplified = True)
    film_viscosity = viscosity(T, Pvap, simplified = True)
    film_prandtl = film_heat_capacity * film_viscosity / film_conductivity
    
    exhaust_steam_density = vapour_density(Pvap, simplified = True)
    exhaust_vaporization_heat = vaporization_enthalpy(Pvap, simplified = True)
    
    num = (g * film_density * (film_density - exhaust_steam_density) * film_conductivity ** 3. * exhaust_vaporization_heat)
    den = film_viscosity * (Tvap - To) * Do
    hext_row1 = 0.729 * (num / den) ** 0.25
    Resext_row1 = 1 / (pi * Do * hext_row1 * L)

    hext = hext_row1 * fNtub_tube
    Resext = Resext_row1 / fNtub_tube
    
    return hext, Resext, exhaust_vaporization_heat
    
    
def calculate_wall(Do, Di, kwall, L):
    
    Reswall = np.log(Do / Di) / (2 * pi * kwall * L)
    
    return Reswall

In [53]:
def calculate_heat(k_tube,
                   cp,
                   Tin,
                   Tout):
    """Calculate the duty"""
    return k_tube * cp * (Tout - Tin)


def calculate_LMTD(T1a, T1b, T2a, T2b):
    """Calculate LMTD"""
    dT1 = T1a - T1b
    dT2 = T2a - T2b
    return (dT1 - dT2) / np.log(dT1/dT2)

def calculate_heat_via_LMTD( UA, LMTD):
    """Calculate heat exchanger via LMTD"""
    return UA * LMTD

def fun(x, Tvap, Tin, Pin, fNtub, ResF):

    f = []
    
    m_ = 0
    
    Pout = x[0]
    
    for i in range(0,Rows):
        
        fNtub_i = fNtub[i]
        
        Tout, Tf_in, Tf_out, Ti_in, Ti_out, To_in, To_out, m_tube = x[(8*i+1):(8*(i+1)+1)]
        
        m_ += m_tube * tubes_per_width

        #### WALL RESISTANCE
        Reswall = calculate_wall(Do, Di, kwall, L)


        #### EXTERNAL RESISTANCE
        To = 0.5 * (To_in + To_out)
        hext, Resext, _ = calculate_ext(Tvap, Pvap, To, fNtub_i, L)


        #### INTERNAL RESISTANCE
        hint, Resint, cp, Pout_, fD_tube, pressure_loss_in_tube, v, Re_tube = calculate_int(
            Tin, Tout, Pin, Pout, m_tube, ep, Di, L
        )

        # Calculate Overall he
        Restotal = Resext + Resint + Reswall + ResF  # W/K

        UA = 1/Restotal  # W/K

        # Wall conduction
        Reswall = np.log(Do / Di) / (2 * pi * kwall * L)

        LMTD = calculate_LMTD(Tvap, Tin, Tvap, Tout)

        Q1 = calculate_heat_via_LMTD( UA, LMTD)

        Q2 = calculate_heat( m_tube, cp, Tin, Tout)

        Q_at_inlet = (Tvap - Tin) / Restotal # W
        Q_at_outlet = (Tvap - Tout) / Restotal # W

                
        f += [
            Pout - Pout_,
            Q1 - Q2,
            To_in - Tvap + Q_at_inlet*Resext,
            Ti_in - To_in + Q_at_inlet*Reswall,
            Tf_in - Ti_in + Q_at_inlet*ResF,
            To_out - Tvap + Q_at_outlet*Resext,
            Ti_out - To_out + Q_at_outlet*Reswall,
            Tf_out - Ti_out + Q_at_outlet*ResF
        ]
        
    f.append(m - m_)
    
    out = np.array(f)
    
    #print(np.sum(out**2))
    
    return out

def solve(x0, *args):
    sol = root(fun, x0, args=args)
    return  sol.x

In [54]:
Tout = Tout_mean_expected * np.ones(Rows) # K
Tf_in = 1*Tin + 0*Tout + 0*Tvap
Tf_out = 0*Tin + 1*Tout + 0*Tvap
Ti_in = 1*Tin + 0*Tout + 0*Tvap
Ti_out = 0*Tin + 1*Tout + 0*Tvap
To_in = 0.5*Tin + 0*Tout + 0.5*Tvap
To_out = 0*Tin + 0.5*Tout + 0.5*Tvap
m_tube = 0*Tout + m/n_tubes

In [55]:
x0 = [Pout_expected]

for i in range(0, Rows):
     x0 +=[Tout[i], Tf_in[i], Tf_out[i], Ti_in[i], Ti_out[i], To_in[i], To_out[i], m_tube[i]]

res0 = fun(x0, Tvap, Tin, Pin, fNtub, ResF)
x = solve(x0, Tvap, Tin, Pin, fNtub, ResF )
res = fun(x, Tvap, Tin, Pin, fNtub, ResF)

Pout = x[0]
for i in range(0,Rows):
    Tout[i], Tf_in[i], Tf_out[i], Ti_in[i], Ti_out[i], To_in[i], To_out[i], m_tube[i] = x[(8*i+1):(8*(i+1)+1)]    
    
print("Initial residue", np.sum(res0**2))
print("Final residue", np.sum(res**2))

Initial residue 370335049.4312819
Final residue 2.219529599054847e-06


In [56]:
results = []


for i in range(0,Rows):
    
    Tf = 0.5 * (Tf_in[i] + Tf_out[i])
    Ti = 0.5 * (Ti_in[i] + Ti_out[i])
    To = 0.5 * (To_in[i] + To_out[i])

    
    #### WALL RESISTANCE
    Reswall = calculate_wall(Do, Di, kwall, L)

    
    #### EXTERNAL RESISTANCE
    hext, Resext, exhaust_vaporization_heat = calculate_ext(Tvap, Pvap, To, fNtub[i], L)


    #### INTERNAL RESISTANCE
    hint, Resint, cp, Pout_, fD_tube, pressure_loss_in_tube, v, Re_tube = calculate_int(
        Tin, Tout[i], Pin, Pout, m_tube[i], ep, Di, L
    )

    # Calculate Overall he
    Restotal = Resext + Resint + Reswall + ResF  # W/K

    UA = 1/Restotal  # W/K

    # Calculate LMTD
    LMTD = calculate_LMTD(Tvap,Tin,Tvap,Tout[i])
    Q = calculate_heat_via_LMTD(UA, LMTD)

    results.append({
        "UA": UA,
        "LMTD": LMTD,
        "Q": Q,
        "hint": hint,
        "hext": hext,
        "Restotal": Restotal, # W/K
        "Resint": Resint, # W/K
        "Resext": Resext, # W/K
        "Reswall": Reswall, # W/K
        "Tf_in": Tf_in[i],
        "Ti_in": Ti_in[i],
        "To_in": To_in[i],
        "Tf_out": Tf_out[i],
        "Ti_out": Ti_out[i],
        "To_out": To_out[i],
        "Tf": Tf,
        "Ti": Ti,
        "To": To,
        "Tin": Tin,
        "Tout": Tout[i],
        "Pin": Pin,
        "Pout": Pout,
        "Df": Df,
        "Pout": Pout,
        "v": v,
        "Re": Re_tube,
        "m_tube": m_tube[i],
        "kvap": Q/exhaust_vaporization_heat*tubes_per_width

    })

    
    if i in (0, Rows-1):

        print("")
        print(" -- ROW {} --".format(i))

        results_i = results[i]

        print("Restotal is {number:.{digits}f} W/(K)".format(number=results_i["Restotal"],digits=6))
        print("The row external convection coefficient is {number:.{digits}f} W/(K*m2)".format(number=results_i["hext"],digits=2))
        print("The row internal convection coefficient is {number:.{digits}f} W/(K*m2)".format(number=results_i["hint"],digits=2))

        print("The last row calculated internal resistance x length is {number:.{digits}f} K/W".format(number=results_i["Resint"],digits=6))
        print("The last row calculated external resistance x length is {number:.{digits}f} K/W".format(number=results_i["Resext"],digits=6))
        print("LMTD is {number:.{digits}f} K".format(number=results_i["LMTD"],digits=2))
        print("The heat rate per tube is {number:.{digits}f} W".format(number=results_i["Q"],digits=2))
        print("U*A per tube is {number:.{digits}f} W/K".format(number=results_i["UA"],digits=2))

        print("ResF is {number:.{digits}f} K/W".format(number=ResF,digits=6))
        print("Reswall is {number:.{digits}f} K/W".format(number=results_i["Reswall"],digits=6))
        print("Resint is {number:.{digits}f} K/W".format(number=results_i["Resint"],digits=6))
        print("Resext is {number:.{digits}f} K/W".format(number=results_i["Resext"],digits=6))

        print("Tf is {number:.{digits}f} K".format(number=results_i["Tf"],digits=2))
        print("Ti is {number:.{digits}f} K".format(number=results_i["Ti"],digits=2))
        print("To is {number:.{digits}f} K".format(number=results_i["To"],digits=2))
        print("Tvap is {number:.{digits}f} K".format(number=Tvap,digits=2))
        print("Cooling water inlet temperature is {number:.{digits}f} K".format(number=Tin,digits=2))
        print("Cooling water outlet temperature is {number:.{digits}f} W/K".format(number=results_i["Tout"],digits=2))
        
        print("Water mean velocity is {number:.{digits}f} m/s.".format(number=results_i["v"], digits=2))
        print("Inlet Pressure is {number:.{digits}f} Pa.".format(number=results_i["Pin"], digits=2))
        print("Outlet Pressure : ", results_i["Pout"])
        print("Tube mass flowrate (kg/s) : ", results_i["m_tube"])


 -- ROW 0 --
Restotal is 0.000304 W/(K)
The row external convection coefficient is 13325.81 W/(K*m2)
The row internal convection coefficient is 6419.56 W/(K*m2)
The last row calculated internal resistance x length is 0.000127 K/W
The last row calculated external resistance x length is 0.000055 K/W
LMTD is 11.90 K
The heat rate per tube is 39194.72 W
U*A per tube is 3293.66 W/K
ResF is 0.000060 K/W
Reswall is 0.000061 K/W
Resint is 0.000127 K/W
Resext is 0.000055 K/W
Tf is 298.62 K
Ti is 301.34 K
To is 304.08 K
Tvap is 306.58 K
Cooling water inlet temperature is 284.73 K
Cooling water outlet temperature is 301.02 W/K
Water mean velocity is 1.40 m/s.
Inlet Pressure is 200000.00 Pa.
Outlet Pressure :  179557.37484283876
Tube mass flowrate (kg/s) :  0.5747415238687366

 -- ROW 30 --
Restotal is 0.000375 W/(K)
The row external convection coefficient is 5909.76 W/(K*m2)
The row internal convection coefficient is 6336.92 W/(K*m2)
The last row calculated internal resistance x length is 0.0001

In [58]:
# Calculating final results
Q_total = 0
kvap_calc = 0
Tout_mean = 0
Pout_mean = Pout
m_tube_aux = 0
for i in range(0, Rows):
    Q_total += results[i]["Q"]*tubes_per_width
    kvap_calc += results[i]["kvap"]
    Tout_mean += results[i]["Tout"]*results[i]["m_tube"]
    m_tube_aux += results[i]["m_tube"]
Tout_mean = Tout_mean/m_tube_aux
    
print("The total steam flowrate is {number:.{digits}f} kg/s".format(number=kvap_calc, digits=2))
print("The total heat load of condensation is {number:.{digits}f} MW".format(number=1e-6*Q_total, digits=2))
print("The cooling water CALCULATED mean outlet temperature is {number:.{digits}f} K".format(number=Tout_mean,digits=2))
print("The cooling water EXPECTED mean outlet temperature is {number:.{digits}f} K".format(number=Tout_mean_expected,digits=2))
print("Cooling Water outlet temp deviation is {number:.{digits}f} %".format(number=(Tout_mean-Tout_mean_expected)/(Tout_mean_expected-273.15)*100,digits=2))
print("Steam temp deviation is {number:.{digits}f} %".format(number=(Tvap-Tvap_expected)/(Tvap_expected - 273.15)*100,digits=2))

The total steam flowrate is 101.05 kg/s
The total heat load of condensation is 244.71 MW
The cooling water CALCULATED mean outlet temperature is 299.88 K
The cooling water EXPECTED mean outlet temperature is 299.88 K
Cooling Water outlet temp deviation is -0.01 %
Steam temp deviation is -2.54 %


## Simulations 

### Initial Estimates

In [39]:
Nelements = 15

def prepare_initial_estimate(results, key1, key2):
    return np.array([list(np.linspace(result[key1], result[key2], Nelements)) for result in results]).T.tolist()


data = {
    "node_A": {
        "kind": "node",
        "module": "models.source",
        "class": "Source",
        "steady_state": True,
        "specifications": {
            "w": m,
        },
        "parameters": {
            "Text": Tin,
            "Pext": Pin,
            "x": 0.0,
            "y": 0.0,
            "z": 0.0
        },
        "initial_guess": {
            "w": m,
            "P": Pin,
            "T": Tin

        }
    },
    "condenser": {
        "kind": "edge",

        "module": "models.fixed_external_temperature_tube_arrange",
        "class": "FixedExternalTemperatureTubeArrange",

        #"module": "models.tube_arrange",
        #"class": "TubeArrange",

        "from": "node_A",
        "to": "node_B",
        "domains": {
            "x": {
                "initial": 0.0,
                "final": 1.0,
                "N": Nelements
            },
            "y": {
                "N": Rows
            },

        },
        "states": {
        },
        "parameters": {
            "Di": Di,
            "tetha": 0.0,
            "L": L,
            "epw": ep,
            "Klb": 0.0,
            "Kub": 0.0,
            "Npipes": n_tubes/Rows,
            "kwall": kwall,
            "Do": Do,
            "ResF": ResF

        },
        "steady_state": True,
        "specifications": {
            "To": prepare_initial_estimate(results, "To_in", "To_out"),
        },
        "initial_guess": {
            "ep": ep,
            "Re": prepare_initial_estimate(results, "Re", "Re"),
            "D": Di,
            "v": prepare_initial_estimate(results, "v", "v"),
            "k": prepare_initial_estimate(results, "m_tube", "m_tube"),
            "T": prepare_initial_estimate(results, "Tin", "Tout"),
            "P": prepare_initial_estimate(results, "Pin", "Pout"),
            "hint": prepare_initial_estimate(results, "hint", "hint"),
            "Ti": prepare_initial_estimate(results, "Ti_in", "Ti_out"),

        }
    },
    "node_B": {
        "kind": "node",
        "module": "models.sink",
        "class": "Sink",
        "steady_state": True,
        "specifications": {
            "P": Pout,
        },
        "parameters": {
            "Text": Tout_mean,
            "Pext": Pout,
            "x": 0.0,
            "y": 0.0,
            "z": 0.0
        },
        "initial_guess": {
            "w": m,
            "P": Pout,
            "T": Tout_mean
        }
    },
}

with open("colesoncove_pre.json", 'w') as outfile:
    json.dump(data, outfile, indent=4)
    
simulate.main(
    name="colesoncove_pre",
    input="colesoncove_pre.json",
    output="colesoncove_pre.out.json",
    reporting_interval=1,
    time_horizon=1,
    relative_tolerance=1e-3,
    MaxStep=1,
    MaxNumSteps=1000000,
)

Creating the system...        
The system created successfully.
 0.00% ETA: [00.0s]Starting the initialization of the system... Done.
 0.00% ETA: [00.0s]Integrating from [0] to [1] ...
 0.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]The simulation has finished successfully!
 100.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]                             Time (s)   Rel.time (%)    Count
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]Simulation:                   
 100.00% ETA: [00.0s]  Initialization                2.126              -        -
 100.00% ETA: [00.0s]  Solve initial                22.819          42.35        -
 100.00% ETA: [00.0s]  Integration                  31.061          57.65        -
 100.00% ETA: [00.0s]  Integ. + solve init.         53.881         100.00        -
 100.00% ETA: [

### Simulation

In [40]:
Nelements = 15

def prepare_initial_estimate(results, key1, key2):
    return np.array([list(np.linspace(result[key1], result[key2], Nelements)) for result in results]).T.tolist()
    

data = {
    "node_A": {
        "kind": "node",
        "module": "models.source",
        "class": "Source",
        "steady_state": True,
        "specifications": {
            "w": m,
        },
        "parameters": {
            "Text": Tin,
            "Pext": Pin,
            "x": 0.0,
            "y": 0.0,
            "z": 0.0
        },
        "initial_guess": {
            "w": m,
            "P": Pin,
            "T": Tin

        }
    },
    "condenser": {
        "kind": "edge",

        "module": "models.external_film_condensation_tube_arrange",
        "class": "ExternalFilmCondensationTubeArrange",

        "from": "node_A",
        "to": "node_B",
        "domains": {
            "x": {
                "initial": 0.0,
                "final": 1.0,
                "N": Nelements
            },
            "y": {
                "N": Rows
            },

        },
        "states": {
            'stnRegulator': 'Closed',
            'stnShellPressure': 'Variable'
        },
        "parameters": {
            "Di": Di,
            "tetha": 0.0,
            "L": L,
            "epw": ep,
            "Klb": 0.0,
            "Kub": 0.0,
            "Npipes": n_tubes/Rows,
            "kwall": kwall,
            "Do": Do,
            "PextH": 100000,
            "PextSP": -1,
            "Pext0": Pvap,
            "Vext": Vext,
            "kvap": kvap_calc,
            "fNtub": list(fNtub),
            "ResF": ResF
            #"Text": Tvap,
        },
        "steady_state": True,
        "specifications": {
            #"kcond": 0.0,
            #"Pext": Pvap,
            #"Text": Tvap,
            #"wext": 0.0,
            #"To": prepare_initial_estimate(results, "To_in", "To_out"),
        },
        "initial_guess": {
            "ep": ep,
            "Re": prepare_initial_estimate(results, "Re", "Re"),
            "D": Di,
            "v": prepare_initial_estimate(results, "v", "v"),
            "k": prepare_initial_estimate(results, "m_tube", "m_tube"),
            "T": prepare_initial_estimate(results, "Tin", "Tout"),
            "P": prepare_initial_estimate(results, "Pin", "Pout"),
            "hint": prepare_initial_estimate(results, "hint", "hint"),
            "hext": prepare_initial_estimate(results, "hext", "hext"),
            "Pext": Pvap,
            "Text": Tvap,
            "Ti": prepare_initial_estimate(results, "Ti_in", "Ti_out"),
            #"Qout": prepare_initial_estimate(results, "Qout_in", "Qout_out"),
            "To": prepare_initial_estimate(results, "To_in", "To_out"),
            #"kcond": kvap_calc,
            #"wext": 0.0,
            #"Resistance": prepare_initial_estimate(results, "Restotal", "Restotal"),

        }
    },
    "node_B": {
        "kind": "node",
        "module": "models.sink",
        "class": "Sink",
        "steady_state": True,
        "specifications": {
            "P": Pout,
        },
        "parameters": {
            "Text": Tout_mean,
            "Pext": Pout,
            "x": 0.0,
            "y": 0.0,
            "z": 0.0
        },
        "initial_guess": {
            "w": m,
            "P": Pout,
            "T": Tout_mean
        }
    },
}

with open("colesoncove.json", 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [41]:
simulate.main(
    name="colesoncove",
    input="colesoncove.json",
    output="colesoncove.out.json",
    initial_condition="colesoncove_pre.out.json",
    reporting_interval=10,
    time_horizon=10,
    relative_tolerance=1e-9,
    MaxStep=1,
    MaxNumSteps=1000000,
)

Creating the system...        
The system created successfully.
 0.00% ETA: [00.0s]Starting the initialization of the system... Done.
 0.00% ETA: [00.0s]Integrating from [0] to [10] ...
 0.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]The simulation has finished successfully!
 100.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]                             Time (s)   Rel.time (%)    Count
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]Simulation:                   
 100.00% ETA: [00.0s]  Initialization                2.107              -        -
 100.00% ETA: [00.0s]  Solve initial                17.590          25.87        -
 100.00% ETA: [00.0s]  Integration                  50.416          74.13        -
 100.00% ETA: [00.0s]  Integ. + solve init.         68.006         100.00        -
 100.00% ETA: 

## Results

In [42]:
with open("colesoncove.out.json") as f:
    json_data=f.read()
output = json.loads(json_data)

In [43]:
Tout_sim = output["node_B.T"]["Values"][-1]
Tvap_sim = output["condenser.Text"]["Values"][-1]

print("Simulated Steam Pressure {number:.{digits}f} Pa".format(
    number=output["condenser.Pext"]["Values"][-1],digits=6)
     )
print("Simulated Steam Temperature {number:.{digits}f} oC".format(
    number=Tvap_sim-273.15,digits=2)
     )
print("Simulated Cooling Water Inlet Temperature {number:.{digits}f} oC".format(
    number=output["node_A.T"]["Values"][-1]-273.15,digits=2)
     )
print("Simulated Cooling Water Outlet Temperature {number:.{digits}f} oC".format(
    number=Tout_sim-273.15,digits=2)
     )
print("Simulated Steam flowrate {number:.{digits}f} kg/s".format(
    number=output["condenser.kvap"]["Values"][-1],digits=2)
     )
print("Simulated Steam Pressure {number:.{digits}f} Pa".format(
    number=sum(np.array(output["condenser.Qtotal"]["Values"][-1]))/1e6,digits=2)
     )

print("Cooling Water outlet temp deviation is {number:.{digits}f} %".format(number=(Tout_sim-Tout_mean_expected)/(Tout_mean_expected-273.15)*100,digits=2))
print("Steam temp deviation is {number:.{digits}f} %".format(number=(Tvap_sim-Tvap_expected)/(Tvap_expected - 273.15)*100,digits=2))

Simulated Steam Pressure 5107.946245 Pa
Simulated Steam Temperature 33.35 oC
Simulated Cooling Water Inlet Temperature 11.58 oC
Simulated Cooling Water Outlet Temperature 27.50 oC
Simulated Steam flowrate 101.05 kg/s
Simulated Steam Pressure 244.74 Pa
Cooling Water outlet temp deviation is 2.87 %
Steam temp deviation is -2.76 %


In [21]:
print(output["condenser.T"]["Values"][-1][0])

[284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559, 284.730002838559]


In [22]:
print(output["condenser.T"]["Values"][-1][-1])

[301.5014538739156, 301.50145387391564, 301.0166600645525, 301.01666006455247, 300.8095366873045, 300.80953668730456, 300.6646949538038, 300.6646949538038, 300.55143644307276, 300.5514364430728, 300.4576892475295, 300.4576892475295, 300.3773255927066, 300.37732559270665, 300.30677035255223, 300.3067703525523, 300.24374159872906, 300.24374159872906, 300.18668751402237, 300.18668751402237, 300.134501644644, 300.134501644644, 300.08636552800886, 300.0863655280088, 300.0416556395831, 300.0416556395831, 299.9998853596973, 299.9998853596973, 299.96066718256, 299.96066718256, 299.92368720992124]
